# HPX + Cling + Jupyter

## Docker Instructions
* Frist, install docker and docker-compose on your local resource
* Second, start Docker, e.g. ```sudo service docker start```
* Third, run `curl -LO https://raw.githubusercontent.com/stevenrbrandt/CxxExplorer/master/docker-compose.yml`
* Fourth, edit the docker-compose.yml and change the password to something other than `spoon`
* Fifth, start the notebook by typing `docker-compose up -d`
* Sixth, play with the existing ipynb files or create new ones.
* Seven, save your work! This is an important step. If you simply quit the container, everything you did will be lost. You can copy individual files from the image to local disk as follows: `docker cp cxxex-src-nbk:/home/jovyan/HPX_by_example.ipynb ~/`

In [ ]:
#include <run_hpx.cpp>

In [ ]:
using namespace std;
using namespace hpx;

# What is a (the) Future?

Many ways to get hold of a future, simplest way is to use (std) async:

In [ ]:
int universal_answer() { return 42; }
void deep_thought()
{
  future<int> promised_answer = async(&universal_answer);
  // do other things for 7.5 million years
  cout << promised_answer.get() << endl; // prints 42
}

If we want to do something other than a declaration, use the ".expr" prefix.

In [ ]:
.expr run_hpx([](){ deep_thought(); });

In [ ]:
#define WRAP_EXPR(X) run_hpx([](){X});

In [ ]:
deep_thought();

# Compositional Facilities

In [ ]:
future<string> make_string()
{
    future<int> f1 = async([]()->int { return 123; });
    future<string> f2 = f1.then(
      [](future<int> f) -> string
      {
        return to_string(f.get()); // here .get() won't block
      });
    return f2;
}

In [ ]:
cout << make_string().get() << endl;

In [ ]:
int do_work(hpx::lcos::future<hpx::util::tuple<hpx::lcos::future<int>,
      hpx::lcos::future<std::basic_string<char> > > >& w) {
  // extract the value of the first argument.
  return hpx::get<0>(w.get()).get();
}

future<int> test_when_all()
{
  future<int> future1 = async([]()->int { return 125; });
  future<string> future2 = async([]()->string { return string("hi"); });
  
  auto all_f = when_all(future1,future2);
  
  future<int> result = all_f.then(
    [](auto f)->int {
      return do_work(f);
    });
  return result;
}

In [ ]:
cout << test_when_all().get() << endl;

# Parallel Algorithms
HPX allows you to write loop parallel algorithms in a generic fashion, applying to specify the way in which parallelism is achieved (i.e. threads, distributed, cuda, etc.) through polcies.

In [ ]:
#include <hpx/include/parallel_for_each.hpp>
#include <hpx/parallel/algorithms/transform.hpp>
#include <boost/iterator/counting_iterator.hpp>

In [ ]:
vector<int> v = { 1, 2, 3, 4, 5, 6 };

## Transform
Here we demonstrate the transformation of a vector, and the various mechnanisms by which it can performed in parallel.

In [ ]:
// This parallel tranformation of vector v
// is done using thread parallelism. An
// implicit barrier is present at the end.
hpx::transform (
  hpx::execution::par,
  begin(v), end(v), begin(v),
  [](int i) -> int
  {
    return i+1;  
  });
for(int i : v) cout << i << ",";

In [ ]:
.expr
// This parallel tranformation of vector v
// is done using thread parallelism. There
// is no implicit barrier. Instead, the
// transform returns a future.
auto  f = hpx::transform (
  execution::par (execution::task),
  begin(v), end(v), begin(v),
  [](int i) -> int
  {
    return i+1;  
  });
  // work here...
// wait for the future to be ready.
f.wait();

for(int i : v) cout << i << ",";

In [ ]:
#include <hpx/include/parallel_fill.hpp>
#include <hpx/include/compute.hpp>
#include <hpx/include/parallel_executors.hpp>

In [ ]:
// targets are threads. Possibly, they each have their own locality (although by
// using get_local_targets we'll only get targets on the current locality)
auto host_targets = run_hpx([](){ return hpx::compute::host::get_local_targets(); });

In [ ]:
// List the locality of all targets. These should all be the same.
for(auto host : host_targets)
  cout << host.get_locality() << endl;

In [ ]:
// Get all targets on all localities and store in host_targets.
// This notebook only uses a single locality, so this list should
// be the same as the above.
host_targets.clear();
for (hpx::id_type const& locality : hpx::find_all_localities())
{
    std::vector<hpx::compute::host::target> targets =
        hpx::compute::host::get_targets(locality).get();
    host_targets.insert(end(host_targets),begin(targets),end(targets));
}
for(auto host : host_targets)
  cout << host.get_locality() << endl;

## Other Algorithms
There are a great many algorithms. Here we demonstrate a handful of them.

In [ ]:
typedef hpx::compute::host::block_executor<> executor_type;

In [ ]:
.expr
// Do a fill on the listed targets. We have supplied all of them,
// though we don't need to. Note that the fill algorithm doesn't make sense
// in a distributed setting since vector vd is not distributed.
executor_type exec(host_targets);
std::vector<float> vd;
for(int i=0;i<10;i++) vd.push_back(1.f);
hpx::fill(execution::par.on(exec),vd.begin(),vd.end(),1.0f*getpid());
for(int i=0;i<10;i++) cout << vd[i] << " "; cout << std::endl;

In [ ]:
#include <hpx/parallel/algorithms/reverse.hpp>

In [ ]:
.expr
std::vector<float> vd;
for(int i=0;i<10;i++) vd.push_back(1.f*i);
hpx::reverse(execution::par,vd.begin(),vd.end());
for(int val : vd) cout << val << " ";

In [ ]:
#include <hpx/include/parallel_minmax.hpp>

In [ ]:
.expr
std::vector<float> vd;
for(int i=0;i<10;i++) vd.push_back(1.f*rand());
auto ptr = hpx::max_element(execution::par,vd,std::less<float>());
for(float val : vd) cout << val << " ";
cout << endl << *ptr << endl;

In [ ]:
#include <hpx/execution.hpp>
#include <hpx/include/parallel_executors.hpp>

In [ ]:
int count_async = 0;
struct test_async_executor
{
    typedef hpx::execution::parallel_execution_tag execution_category;

    template <typename F, typename ... Ts> 
    static hpx::future<typename hpx::util::result_of<F&&(Ts&&...)>::type>
    async_execute(F && f, Ts &&... ts) 
    {   
        ++count_async;
        return hpx::async(hpx::launch::async, std::forward<F>(f),
            std::forward<Ts>(ts)...);
    }
};

In [ ]:
// Note that the exact way to specify this trait for an executor is in flux
// and the code here is tied to the specific version of HPX on the test machine.
namespace hpx { namespace traits
{
    template<>
    struct is_two_way_executor<test_async_executor>
      : std::true_type
    {};
}}

In [ ]:
.expr
// This parallel tranformation of vector v
// is done using using distributed parallelism.
// NOTE: Ignore the warning
test_async_executor e;
hpx::transform (
  execution::par.on(e),
  begin(v), end(v), begin(v),
  [](int i) -> int
  {
    return i+1;  
  });
cout << "count=" << count_async << endl;

# Let’s Parallelize It – Adding Real Asynchrony

Here we take a step back. Instead of using a pre-designed parallel operation on a vector, we instead introduce task-level parallelism to an existing program.

Calculate Fibonacci numbers in parallel (1st attempt)

In [ ]:
uint64_t fibonacci(uint64_t n)
{
  // if we know the answer, we return the value
  if (n < 2) return n;
  // asynchronously calculate one of the sub-terms
  future<uint64_t> f = async(launch::async, &fibonacci, n-2);
  // synchronously calculate the other sub-term
  uint64_t r = fibonacci(n-1);
  // wait for the future and calculate the result
  return f.get() + r;
}

In [ ]:
cout << fibonacci(10) << endl;

# Let’s Parallelize It – Introducing Control of Grain Size
Parallel calculation, switching to serial execution below given threshold

In [ ]:
const int threshold = 20;

uint64_t fibonacci_serial(uint64_t n)
{
  if (n < 2) return n;
  uint64_t f1 = fibonacci_serial(n-2);
  uint64_t f2 = fibonacci_serial(n-1);
  return f1 + f2;
}

uint64_t fibonacci2(uint64_t n)
{
  if (n < 2) return n;
  if (n < threshold) return fibonacci_serial(n);
  // asynchronously calculate one of the sub-terms
  future<uint64_t> f = async(launch::async, &fibonacci2, n-2);
  // synchronously calculate the other sub-term
  uint64_t r = fibonacci2(n-1);
  // wait for the future and calculate the result
  return f.get() + r;
}

In [ ]:
cout << fibonacci2(22) << endl;

# Let’s Parallelize It – Apply Futurization
Parallel way, futurize algorithm to remove suspension points

In [ ]:
future<uint64_t> fibonacci3(uint64_t n)
{
  if(n < 2) return make_ready_future(n);
  if(n < threshold) return make_ready_future(fibonacci_serial(n));

  future<uint64_t> f = async(launch::async, &fibonacci3, n-2);
  future<uint64_t> r = fibonacci3(n-1);

  return dataflow(
    [](future<uint64_t> f1, future<uint64_t> f2) {
      return f1.get() + f2.get();
    },
    f, r);
}

In [ ]:
cout << fibonacci3(22).get() << endl;

# Let’s Parallelize It – Unwrap Argument Futures

In [ ]:
using hpx::util::unwrapping;

future<uint64_t> fibonacci4(uint64_t n)
{
  if(n < 2) return make_ready_future(n);
  if(n < threshold) return make_ready_future(fibonacci_serial(n));

  future<uint64_t> f = async(launch::async, &fibonacci4, n-2);
  future<uint64_t> r = fibonacci4(n-1);

  return dataflow(
    unwrapping([](uint64_t f1, uint64_t f2) {
      return f1+f2;
    }),
    f, r);
}

In [ ]:
cout << fibonacci4(22).get() << endl;

### Excercise: Parallelize a sort
Test what you've learned. See if you can speed up the quicksort program below by find a place to:
1. parallelize the code with async
2. use parallel transforms

In [ ]:
#include <unistd.h>
#include <stdlib.h>
#include <iostream>
#include <vector>
#include <functional>

typedef std::vector<int>::iterator viter;

std::function<void(viter, viter)> myqsort;

<h3>Discussion</h3>
We want to define the myqsort function repeatedly, and call it recursively. This is hard to do in C++. So we define it as a std::function&lt;&gt;. There is a slight
awkwardness to this. If you want to call myqsort() with an async function, you have to do it like this:
<pre>
  auto f = hpx::async([&](){ myqsort(a, b); });
</pre>
Not like this
<pre>
  auto f = hpx::async(myqsort, a, b);
</pre>

In [ ]:
myqsort = [](viter first, viter last)->void {
  assert(first <= last);
  ptrdiff_t range = last - first;
  if(range > 1) {
      auto pivot_value = first[rand() % range];
      // Try converting this to use a parallel algorithm
      auto middle1 = std::partition(first, last, [&](auto em) { return em < pivot_value;});
      assert(middle1 <= last);
      auto middle2 = std::partition(middle1, last, [&](auto em) { return em <= pivot_value;});
      assert(middle2 <= last);

      // try invoking this using hpx::async
      myqsort(first,middle1);
      myqsort(middle2,last);
  }
};
vector<int> vv{20};
for(int i=0;i<20;i++) vv.push_back(rand() % 100);
for(int val : vv) cout << val << " ";
cout << endl;
myqsort(begin(vv),end(vv));
for(int val : vv) cout << val << " ";
cout << endl;